# DWH Project: Comprehensive Data Analysis

This notebook performs a full analysis of the three provided CSV files to prepare for the DWH project.

**Goals:**
1.  **Cleaning:** Clean and prepare data for SQL loading.
2.  **Integrity:** Check primary key uniqueness and referential integrity (orphaned records).
3.  **EDA:** Understand the distributions of key dimensions for the 20 analytical queries.
4.  **ETL Simulation:** Simulate the final "Transformed Data" by joining all tables.

## 0. Imports and Data Loading

In [1]:
import pandas as pd
import numpy as np

# Set display options for better output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
try:
    cs_master = pd.read_csv('Data/customer_master_data.csv')
    pd_master = pd.read_csv('Data/product_master_data.csv')
    tr_data = pd.read_csv('Data/transactional_data.csv')
    print("Files loaded successfully from 'Data/' directory.")
except FileNotFoundError:
    print("Files not found in 'Data/' directory. Trying current directory...")
    cs_master = pd.read_csv('customer_master_data.csv')
    pd_master = pd.read_csv('product_master_data.csv')
    tr_data = pd.read_csv('transactional_data.csv')
    print("Files loaded successfully from current directory.")

print("\n--- Customer Master Info ---")
cs_master.info()
print("\n--- Product Master Info ---")
pd_master.info()
print("\n--- Transactional Data Info ---")
tr_data.info()

Files loaded successfully from 'Data/' directory.

--- Customer Master Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5891 entries, 0 to 5890
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  5891 non-null   int64 
 1   Customer_ID                 5891 non-null   int64 
 2   Gender                      5891 non-null   object
 3   Age                         5891 non-null   object
 4   Occupation                  5891 non-null   int64 
 5   City_Category               5891 non-null   object
 6   Stay_In_Current_City_Years  5891 non-null   int64 
 7   Marital_Status              5891 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 368.3+ KB

--- Product Master Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3631 entries, 0 to 3630
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------         

## 1. Data Cleaning and Preparation

Prepare the DataFrames for analysis and SQL loading. This involves dropping junk columns, renaming for SQL compatibility, and setting correct data types.

In [3]:
print("Dropping 'Unnamed: 0' columns...")

cs_master = cs_master.drop(columns=['Unnamed: 0'], errors='ignore')
pd_master = pd_master.drop(columns=['Unnamed: 0'], errors='ignore')
tr_data = tr_data.drop(columns=['Unnamed: 0'], errors='ignore')

print("Columns dropped.")

Dropping 'Unnamed: 0' columns...
Columns dropped.


In [4]:
print("Renaming 'price$' to 'price'...")
pd_master = pd_master.rename(columns={'price$': 'price'})
print(f"Product master columns: {pd_master.columns.tolist()}")

Renaming 'price$' to 'price'...
Product master columns: ['Product_ID', 'Product_Category', 'price', 'storeID', 'supplierID', 'storeName', 'supplierName']


In [5]:
print("Converting 'tr_data.date' to datetime objects...")
tr_data['date'] = pd.to_datetime(tr_data['date'])
print(f"Data type of 'date' column: {tr_data['date'].dtype}")

Converting 'tr_data.date' to datetime objects...
Data type of 'date' column: datetime64[ns]


In [6]:
print("--- Primary Key Uniqueness Check ---")
print(f"Customer_ID is unique in cs_master: {cs_master['Customer_ID'].is_unique}")
print(f"Product_ID is unique in pd_master:  {pd_master['Product_ID'].is_unique}")

print(f"Total cs_master rows: {len(cs_master)}, Unique Customer_IDs: {cs_master['Customer_ID'].nunique()}")
print(f"Total pd_master rows: {len(pd_master)}, Unique Product_IDs:  {pd_master['Product_ID'].nunique()}")

--- Primary Key Uniqueness Check ---
Customer_ID is unique in cs_master: True
Product_ID is unique in pd_master:  True
Total cs_master rows: 5891, Unique Customer_IDs: 5891
Total pd_master rows: 3631, Unique Product_IDs:  3631


## 2. Referential Integrity Analysis (Orphaned Records)

This is the MOST important check for the ETL process. Do all transactions in `tr_data` have a matching customer and product in the master tables? Any transaction that *doesn't* is an "orphaned record" and will fail the join.

In [7]:
print("--- Checking for Orphaned Customer Records ---")
unique_trans_cust = set(tr_data['Customer_ID'].unique())
unique_master_cust = set(cs_master['Customer_ID'].unique())

orphaned_cust_ids = unique_trans_cust - unique_master_cust

if not orphaned_cust_ids:
    print("GOOD: All Customer_IDs in transactions exist in the customer master table.")
else:
    print(f"WARNING: Found {len(orphaned_cust_ids)} orphaned Customer_IDs in transactions.")
    print(f"Example orphans: {list(orphaned_cust_ids)[:5]}")

--- Checking for Orphaned Customer Records ---
GOOD: All Customer_IDs in transactions exist in the customer master table.


In [8]:
print("--- Checking for Orphaned Product Records ---")
unique_trans_prod = set(tr_data['Product_ID'].unique())
unique_master_prod = set(pd_master['Product_ID'].unique())

orphaned_prod_ids = unique_trans_prod - unique_master_prod

if not orphaned_prod_ids:
    print("GOOD: All Product_IDs in transactions exist in the product master table.")
else:
    print(f"WARNING: Found {len(orphaned_prod_ids)} orphaned Product_IDs in transactions.")
    print(f"Example orphans: {list(orphaned_prod_ids)[:5]}")

--- Checking for Orphaned Product Records ---
GOOD: All Product_IDs in transactions exist in the product master table.


## 3. Exploratory Data Analysis (EDA)

Understand the distributions and unique values of the key dimension attributes. This is essential for writing the 20 analytical queries.

### 3.1 Customer Dimension (`cs_master`)

In [9]:
print("--- Customer Dimension Analysis ---")
print("\nDistribution of 'Age' (for Q2, Q4, Q10):")
print(cs_master['Age'].value_counts().sort_index())

print("\nDistribution of 'Occupation' (for Q3, Q5):")
print(cs_master['Occupation'].value_counts().sort_index())

print("\nDistribution of 'City_Category' (for Q2, Q6, Q8):")
print(cs_master['City_Category'].value_counts().sort_index())

print("\nDistribution of 'Gender' (for Q2, Q4, Q7):")
print(cs_master['Gender'].value_counts())

print("\nDistribution of 'Marital_Status' (for Q6):")
print(cs_master['Marital_Status'].value_counts())

print("\nDistribution of 'Stay_In_Current_City_Years' (for Q7):")
print(cs_master['Stay_In_Current_City_Years'].value_counts().sort_index())

--- Customer Dimension Analysis ---

Distribution of 'Age' (for Q2, Q4, Q10):
Age
0-17      218
18-25    1069
26-35    2053
36-45    1167
46-50     531
51-55     481
55+       372
Name: count, dtype: int64

Distribution of 'Occupation' (for Q3, Q5):
Occupation
0     688
1     517
2     256
3     170
4     740
5     111
6     228
7     669
8      17
9      88
10    192
11    128
12    376
13    140
14    294
15    140
16    235
17    491
18     67
19     71
20    273
Name: count, dtype: int64

Distribution of 'City_Category' (for Q2, Q6, Q8):
City_Category
A    1045
B    1707
C    3139
Name: count, dtype: int64

Distribution of 'Gender' (for Q2, Q4, Q7):
Gender
M    4225
F    1666
Name: count, dtype: int64

Distribution of 'Marital_Status' (for Q6):
Marital_Status
0    3417
1    2474
Name: count, dtype: int64

Distribution of 'Stay_In_Current_City_Years' (for Q7):
Stay_In_Current_City_Years
0     772
1    2086
2    1145
3     979
4     909
Name: count, dtype: int64


### 3.2 Product Dimension (`pd_master`)

In [10]:
print("--- Product Dimension Analysis ---")
print("\nDistribution of 'Product_Category' (for Q3, Q5, Q8, Q9):")
print(pd_master['Product_Category'].value_counts())

print("\nUnique Stores, Suppliers, and Categories:")
print(f"Number of unique 'storeName':    {pd_master['storeName'].nunique()} (for Q12, Q13, Q15, Q17)")
print(f"Number of unique 'supplierName': {pd_master['supplierName'].nunique()} (for Q13, Q15, Q17)")

print("\nStatistical Summary of 'price' (for Q1, Q18):")
print(pd_master['price'].describe())

--- Product Dimension Analysis ---

Distribution of 'Product_Category' (for Q3, Q5, Q8, Q9):
Product_Category
Toys                        1047
Health & Beauty              967
Grocery                      493
Patio & Garden               254
Electronics                  152
Household Essentials         119
Baby                         102
Arts, Crafts & Sewing         98
Home & Kitchen                90
Clothing                      88
Office & School Supplies      44
Appliances                    44
Furniture                     35
Books, Movies & Music         30
Pets                          25
Automotive                    25
Pharmacy & OTC                11
Shoes                          3
Sports & Outdoors              2
Jewelry & Accessories          2
Name: count, dtype: int64

Unique Stores, Suppliers, and Categories:
Number of unique 'storeName':    8 (for Q12, Q13, Q15, Q17)
Number of unique 'supplierName': 7 (for Q13, Q15, Q17)

Statistical Summary of 'price' (for Q1, Q18):

### 3.3 Transactions Fact (`tr_data`)

In [11]:
print("--- Transaction Data Time Analysis ---")
print(f"Earliest transaction date: {tr_data['date'].min()}")
print(f"Latest transaction date:   {tr_data['date'].max()}")

# This is critical for queries like Q12 ("for 2017")
print("\nTransaction counts by year:")
print(tr_data['date'].dt.year.value_counts().sort_index())

--- Transaction Data Time Analysis ---
Earliest transaction date: 2015-01-01 00:00:00
Latest transaction date:   2020-12-31 00:00:00

Transaction counts by year:
date
2015    91870
2016    91935
2017    91627
2018    91258
2019    91821
2020    91557
Name: count, dtype: int64


In [12]:
print("--- Transaction Data Measures Analysis ---")
print("\nStatistical Summary of 'quantity' (for Q18):")
print(tr_data['quantity'].describe())

print("\nDistribution of 'quantity':")
print(tr_data['quantity'].value_counts().sort_index())

--- Transaction Data Measures Analysis ---

Statistical Summary of 'quantity' (for Q18):
count    550068.000000
mean          1.998338
std           0.816231
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           3.000000
Name: quantity, dtype: float64

Distribution of 'quantity':
quantity
1    183694
2    183594
3    182780
Name: count, dtype: int64


In [13]:
print("--- 'Hot Key' Analysis (for HYBRIDJOIN efficiency) ---")
print("\nTop 10 most frequent Customer_IDs:")
print(tr_data['Customer_ID'].value_counts().head(10))

print("\nTop 10 most frequent Product_IDs:")
print(tr_data['Product_ID'].value_counts().head(10))

--- 'Hot Key' Analysis (for HYBRIDJOIN efficiency) ---

Top 10 most frequent Customer_IDs:
Customer_ID
1001680    1026
1004277     979
1001941     898
1001181     862
1000889     823
1003618     767
1001150     752
1001015     740
1005795     729
1005831     727
Name: count, dtype: int64

Top 10 most frequent Product_IDs:
Product_ID
P00265242    1880
P00025442    1615
P00110742    1612
P00112142    1562
P00057642    1470
P00184942    1440
P00046742    1438
P00058042    1422
P00145042    1406
P00059442    1406
Name: count, dtype: int64


## 4. Simulating the Final Joined Data (ETL Verification)

This final step simulates the full ETL join process. The resulting `full_df` DataFrame is exactly what your `HYBRIDJOIN` algorithm is meant to produce and load into the DW.

In [14]:
print("Simulating ETL Joins...")

# 1. Join transactions with customers (Stream S + Relation R1)
# We use 'inner' join to discard any orphaned records, as is standard.
merged_df = pd.merge(tr_data, cs_master, on='Customer_ID', how='inner')

# 2. Join result with products (Result + Relation R2)
full_df = pd.merge(merged_df, pd_master, on='Product_ID', how='inner')

print("Joins complete.")
print(f"Original transactions: {len(tr_data)}")
print(f"Transactions after merge: {len(full_df)}")
print(f"Total records lost to orphans: {len(tr_data) - len(full_df)}")

Simulating ETL Joins...
Joins complete.
Original transactions: 550068
Transactions after merge: 550068
Total records lost to orphans: 0


In [15]:
print("Creating the 'Total_Purchase' measure (quantity * price)...")
full_df['Total_Purchase'] = full_df['quantity'] * full_df['price']
print("Measure created.")

Creating the 'Total_Purchase' measure (quantity * price)...
Measure created.


In [16]:
print("--- Final 'Transformed Data' Schema ---")
full_df.info()

print("\n--- Final 'Transformed Data' Head ---")
print(full_df.head())

--- Final 'Transformed Data' Schema ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   orderID                     550068 non-null  int64         
 1   Customer_ID                 550068 non-null  int64         
 2   Product_ID                  550068 non-null  object        
 3   quantity                    550068 non-null  int64         
 4   date                        550068 non-null  datetime64[ns]
 5   Gender                      550068 non-null  object        
 6   Age                         550068 non-null  object        
 7   Occupation                  550068 non-null  int64         
 8   City_Category               550068 non-null  object        
 9   Stay_In_Current_City_Years  550068 non-null  int64         
 10  Marital_Status              550068 non-null  int64         
 11 